## Realtime object detection using tensorflow2 and opencv-python

### Prerequisites and Installs

#### Create a virtual working environment

#### Linux
Run `python3 -m venv env`
Actiate environment `source env/bin/activate`

#### Windows
Run `python -m venv env`
Activate `source env/scripts/activate`

#### Install the libraries and modules
The following libraries need to be install for the project to run

- tensorflow
- opencv-python
- tensorflow_hub
- pandas

The packages are also available in the requirements.txt file and can be installed using the command `pip install -r requirements.txt`


## The libraries can be installed by the following commands when using google colab

!pip install tensorflow
!pip install opencv-python
!pip install tensorflow_hub
!pip install pandas

## Folder tree
The Root project tree is as follows

![Root folder tree](./input/tree.png)

The folder consists of 4 directories and 3 files

#### Dirs
- env: The virtualenv directory
- input: Contains labels.csv
- models: Contains the saved_model.pb
- output: Contains the sample output screenshots

#### Files
- main.ipynb : A notebook file that documents and demostrates the code line by line
- main.py : A python script version of the notebook file
- requirements.txt : a pip dependancy saved file

## Import the relevant libraries installed

In [ ]:
import tensorflow_hub as hub
import cv2
import numpy
import tensorflow as tf
import pandas as pd

### Load models


In [ ]:
# Load models
detector = hub.load("./models")
labels = pd.read_csv('./input/labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

## Enable computer camera

In [ ]:
cap = cv2.VideoCapture(0)

### Set window size

In [ ]:
width = 1080
height = 720


### Capture frames continually and show the rusulting detected object frames

In [ ]:
while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()
    
    #Resize to respect the input_shape
    inp = cv2.resize(frame, (width , height ))

    #Convert img to RGB
    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

    #Is optional but i recommend (float convertion and convert img to tensor image)
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)
    
    boxes, scores, classes, num_detections = detector(rgb_tensor)
    
    pred_labels = classes.numpy().astype('int')[0]
    
    pred_labels = [labels[i] for i in pred_labels]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]
   
   #loop throughout the detections and place a box around it  
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
            
        score_txt = f'{100 * round(score,0)}'
        img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),1)      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_boxes,label,(xmin, ymax-10), font, 0.5, (255,0,0), 1, cv2.LINE_AA)
        cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 0.5, (255,0,0), 1, cv2.LINE_AA)



    #Display the resulting frame
    cv2.imshow('black and white',img_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## The output
The following are the screenshots of the program in action

![Output Frames Screenshot](./output/1.png)
![Output Frames Screenshot](./output/2.png)
![Output Frames Screenshot](./output/3.png)
![Output Frames Screenshot](./output/4.png)